### Imports

In [1]:

import networkx as nx
import numpy as np
import random, math
import graphviz
import functools, collections, operator
import seaborn as sns

from scipy.stats import shapiro
import scipy.stats as scs
from scipy.stats import describe

import plotly.figure_factory as ff

from scipy.stats import norm
import scipy.stats as stats
from scipy.stats import pearsonr

import os
import time

import pylab

from statsmodels.graphics.gofplots import qqplot

#import kaleido

import pandas as pd


import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm

import plotly.graph_objects as go
from pyvis.network import Network
from networkx.drawing.nx_agraph import graphviz_layout
import plotly.express as px
from plotly._subplots import make_subplots
import plotly

import statsmodels.formula.api as sm

from stargazer.stargazer import Stargazer


# import models
from classes.jackson_model import JacksonSimulationV2
from classes.network_drawing import plotly_sim_drawing
from classes.plotly_drawings import plotly_lines


def MTO_simulator(initial_n=0, T=100, m=10, pm_o=1, n=10, pn_o=1, p_SES_high=0.5, list_rho_values=[1], list_eps_values=[1], num_sim_per_param=1, num_MTO_per_sim=10):

    total_sims = 0
    list_sim_stats = []
    n_sims = len(list_eps_values) * len(list_rho_values) * num_sim_per_param * num_MTO_per_sim
    t0 = time.time()

    print(f'Running {n_sims} simulations over {len(list_eps_values) * len(list_rho_values)} parameter combinations')

    # iterate over eps values
    for i in range(len(list_eps_values)):

        # iterate over rho values
        for j in range(len(list_rho_values)):

            # create a network and run MTO sims
            for k in range(num_sim_per_param):
                # create network
                network_sim = JacksonSimulationV2(initial_n, T, m, pm_o, n, pn_o, p_SES_high, biased=True, epsilon = list_eps_values[i], rho=list_rho_values[j])

                # run MTO simulations
                df_MTO_sim = network_sim.MTO_sim_many(network_sim.graph_history[-1], n_sims=num_MTO_per_sim)


                # calculate network-wide characteristics
                SES_assortativity = nx.assortativity.attribute_assortativity_coefficient(network_sim.graph_history[-1], 'SES_High')
                H_Share_H, H_Share_L = network_sim.helper_functions().average_neighbour_type_per_SES(network_sim.graph_history[-1])
                Network_Degree = sum(dict(network_sim.graph_history[-1].degree()).values()) / len(network_sim.graph_history[-1])
                Network_Clustering = nx.average_clustering(network_sim.graph_history[-1])

                list_network_chars = [SES_assortativity, H_Share_H, H_Share_L, Network_Degree, Network_Clustering]
                list_network_char_names = ['Network_SES_assortativity', 'Network_H_Share_H', 'Network_H_Share_L', 'Network_Degree', 'Network_Clustering']

                # add the values to the df
                for n in range(len(list_network_chars)):
                    df_MTO_sim[list_network_char_names[n]] = list_network_chars[n]

                # append to list
                list_sim_stats.append(df_MTO_sim)
                print('\r', f'Finished simulation {k}/{num_sim_per_param} for combination {total_sims}/{len(list_eps_values)*len(list_rho_values)}', end='')
        
            total_sims += 1

    delta_t = round((time.time()-t0)/60, 2)

    print('\r', f'Finished {n_sims} simulations in {delta_t} minutes, average time: {delta_t/n_sims} min/sim', end='')
    df_MTO_output = pd.concat(list_sim_stats).reset_index(drop=True)

    # add predicted values
    df_MTO_output['Predicted_H_Share'] = 1/((1+df_MTO_output['epsilon'])*(df_MTO_output['rho']))
    df_MTO_output['Predicted_Exposure'] = 1/(df_MTO_output['p_SES_high']*(1+df_MTO_output['epsilon']))
    df_MTO_output['Predicted_Friend_Bias'] = (df_MTO_output['epsilon']*(df_MTO_output['rho']-1))/(df_MTO_output['epsilon']*df_MTO_output['rho']+1)


    return df_MTO_output


### Showcases

In [12]:

# showcases
showcase_sim_no_bias = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=False, rho=1, epsilon=1)
showcase_sim_expo = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=True, rho=1, epsilon=1.75)
showcase_sim_fb = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=True, rho=1.75, epsilon=1)
showcase_sim_expo_fb = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=True, rho=1.5, epsilon=1.5)

showcase_sims = [showcase_sim_no_bias, showcase_sim_expo, showcase_sim_fb, showcase_sim_expo_fb]

showcase_plots = [plotly_sim_drawing.plotly_draw(plotly_sim_drawing(), showcase_sim, layout='spring', draw_largest_CC=True, title=None) for showcase_sim in showcase_sims]





### Rho and Epsilon distributions

In [84]:

'''DESCRIPTIVE STATISTICS'''
df_eps_rho_chetty = pd.read_csv('EpsRhoCalcs.csv')[['county', 'county_name', 'exposure_grp_mem_county', 'exposure_grp_mem_high_county', 'bias_grp_mem_county', 'bias_grp_mem_high_county', 'Epsilon', 'EpsilonHigh', 'Rho', 'RhoHigh']].dropna()

df_eps_rho_chetty = df_eps_rho_chetty.apply(pd.to_numeric, errors='ignore')


df_chetty_summ = df_eps_rho_chetty[['Epsilon', 'Rho']].describe().round(4)
#df_chetty_summ.loc['skew'] = df_chetty_summ.skew()
#df_chetty_summ.to_latex('data.tex')

#Stargazer(df_chetty_summ)
#df_eps_rho_chetty.skew()

eps_all = list(df_eps_rho_chetty['Epsilon'].values) + list(df_eps_rho_chetty['EpsilonHigh'].values)
rho_all = list(df_eps_rho_chetty['Rho'].values) + list(df_eps_rho_chetty['RhoHigh'].values)

print(pearsonr(eps_all, rho_all))

df_eps_rho_chetty.describe()



df_eps_rho_all = pd.DataFrame({'Epsilon': eps_all, 'Rho': rho_all})


px.histogram(df_eps_rho_all[(df_eps_rho_all['Epsilon'] <= df_eps_rho_all['Epsilon'].quantile(0.95)) & df_eps_rho_all['Epsilon'] >= df_eps_rho_all['Epsilon'].quantile(0.05)])

len([i for i in rho_all if (i > np.quantile(rho_all, 0.05)) & (i < np.quantile(rho_all, 0.95))])


PearsonRResult(statistic=-0.3091118936995412, pvalue=1.566535538239077e-133)


5420

In [61]:
px.histogram(df_eps_rho_all)

In [107]:

print(df_eps_rho_all['Rho'].quantile(0.95), df_eps_rho_all['Rho'].quantile(0.05))

df_eps_rho_chetty_var = df_eps_rho_all[(df_eps_rho_all['Rho'] <= 1.85) & (df_eps_rho_all['Rho'] >= 1.05)]

#calculate mean and variance
mu, std = norm.fit(df_eps_rho_chetty_var['Rho'])

# Shapiro Wilk test (want p<0.05)
stat, p = shapiro(df_eps_rho_chetty_var['Rho'])

# count share below 1
below_one_share = len([x for x in list(df_eps_rho_chetty_var['Rho']) if x < 1]) / len(list(df_eps_rho_chetty_var['Rho']))

print(mu, std, stat, p, below_one_share)


1.8502750795999996 1.0014619245999998
1.3306323072692001 0.19917717605546614 0.9420077204704285 1.1958400834855124e-40 0.0


c:\Programming\Python\lib\site-packages\scipy\stats\_morestats.py:1800: UserWarning:

p-value may not be accurate for N > 5000.



In [109]:

'''FIGURES'''

# import the data, convert
#df_eps_rho_chetty = pd.read_csv('EpsRhoCalcs.csv')[['county', 'county_name', 'exposure_grp_mem_county', 'exposure_grp_mem_high_county', 'bias_grp_mem_county', 'bias_grp_mem_high_county', 'Epsilon', 'EpsilonHigh', 'Rho', 'RhoHigh']].dropna()
#df_eps_rho_chetty = df_eps_rho_chetty.apply(pd.to_numeric, errors='ignore')

df_eps_rho_chetty = df_eps_rho_all


vars = ['Epsilon', 'Rho']# 'EpsilonHigh', 'Rho', 'RhoHigh']
figs = []

for var in vars:
    # Winsorize at the 0.5% level
    q05 = df_eps_rho_chetty[var].quantile(0.95)
    q95 = df_eps_rho_chetty[var].quantile(0.05)
    print(q05, q95)

    df_eps_rho_chetty_var = df_eps_rho_chetty[(df_eps_rho_chetty[var] <= q95) & (df_eps_rho_chetty[var] >= q05)]


    #calculate mean and variance
    mu, std = norm.fit(df_eps_rho_chetty_var[var])

    # Shapiro Wilk test (want p<0.05)
    stat, p = shapiro(df_eps_rho_chetty_var[var])

    # count share below 1
    below_one_share = len([x for x in list(df_eps_rho_chetty_var[var]) if x < 1]) / len(list(df_eps_rho_chetty_var[var]))

    fig_title = f'{var} density plot<br>Winsorised at 0.5% level<br>Normal distribution: mean={round(mu,3)}, se={round(std, 3)}<br>Shapiro-Wilk stat and prob: {format(stat, ".3f")}, {"{:.1e}".format(p)}<br>Prob <1: {round(below_one_share, 3)}'

    # Create a histogram of the column
    fig = px.histogram(df_eps_rho_chetty, x=var, nbins=100, opacity=0.9, marginal=None, histnorm='probability density', title=fig_title, width=600, height=600)#, color=df_eps_rho_chetty[var + 'Color'])

    # Add a normal distribution curve to the histogram
    x_axis = np.linspace(df_eps_rho_chetty[var].min(), df_eps_rho_chetty[var].max(), 100)
    y_axis = norm.pdf(x_axis, mu, std)
    curve = go.Scatter(x=x_axis, y=y_axis, mode='lines', line=dict(width=2, color='black'), name=f'Normal Distribution: ', showlegend=False)
    fig.add_trace(curve)

    figs.append(fig)

for var in vars:
    stat, p = shapiro(df_eps_rho_chetty[var])
    print(f'{var} has a SW stat of {format(stat, ".3f")} with p-value {"{:.1e}".format(p)}')

figs[1]


2.5720635710499984 0.57980565985


c:\Programming\Python\lib\site-packages\scipy\stats\_continuous_distns.py:386: RuntimeWarning:

Mean of empty slice.

c:\Programming\Python\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

c:\Programming\Python\lib\site-packages\scipy\stats\_continuous_distns.py:391: RuntimeWarning:

Mean of empty slice.



ValueError: Data must be at least length 3.

# New stuff

### General simulation code

In [3]:

# runs a lot of MTO simulations for a givne list of epsilon and rho values, returning a df
def MTO_simulator(initial_n=0, T=100, m=10, pm_o=1, n=10, pn_o=1, p_SES_high=0.5, list_rho_values=[1], list_eps_values=[1], num_sim_per_param=1, num_MTO_per_sim=10):

    total_sims = 0
    list_sim_stats = []
    n_sims = len(list_eps_values) * len(list_rho_values) * num_sim_per_param * num_MTO_per_sim
    t0 = time.time()

    print(f'Running {n_sims} simulations over {len(list_eps_values) * len(list_rho_values)} parameter combinations')

    # iterate over eps values
    for i in range(len(list_eps_values)):

        # iterate over rho values
        for j in range(len(list_rho_values)):

            # create a network and run MTO sims
            for k in range(num_sim_per_param):
                # create network
                network_sim = JacksonSimulationV2(initial_n, T, m, pm_o, n, pn_o, p_SES_high, biased=True, epsilon = list_eps_values[i], rho=list_rho_values[j])

                # run MTO simulations
                df_MTO_sim = network_sim.MTO_sim_many(network_sim.graph_history[-1], n_sims=num_MTO_per_sim)


                # calculate network-wide characteristics
                SES_assortativity = nx.assortativity.attribute_assortativity_coefficient(network_sim.graph_history[-1], 'SES_High')
                H_Share_H, H_Share_L = network_sim.helper_functions().average_neighbour_type_per_SES(network_sim.graph_history[-1])
                Network_Degree = sum(dict(network_sim.graph_history[-1].degree()).values()) / len(network_sim.graph_history[-1])
                Network_Clustering = nx.average_clustering(network_sim.graph_history[-1])

                list_network_chars = [SES_assortativity, H_Share_H, H_Share_L, Network_Degree, Network_Clustering]
                list_network_char_names = ['Network_SES_assortativity', 'Network_H_Share_H', 'Network_H_Share_L', 'Network_Degree', 'Network_Clustering']

                # add the values to the df
                for n in range(len(list_network_chars)):
                    df_MTO_sim[list_network_char_names[n]] = list_network_chars[n]

                # append to list
                list_sim_stats.append(df_MTO_sim)
                print('\r', f'Finished simulation {k}/{num_sim_per_param} for combination {total_sims}/{len(list_eps_values)*len(list_rho_values)}', end='')
        
            total_sims += 1

    delta_t = round((time.time()-t0)/60, 2)

    print('\r', f'Finished {n_sims} simulations in {delta_t} minutes, average time: {delta_t/n_sims} min/sim', end='')
    df_MTO_output = pd.concat(list_sim_stats).reset_index(drop=True)

    # add predicted values
    df_MTO_output['Predicted_H_Share'] = 1/((1+df_MTO_output['epsilon'])*(df_MTO_output['rho']))
    df_MTO_output['Predicted_Exposure'] = 1/(df_MTO_output['p_SES_high']*(1+df_MTO_output['epsilon']))
    df_MTO_output['Predicted_Friend_Bias'] = (df_MTO_output['epsilon']*(df_MTO_output['rho']-1))/(df_MTO_output['epsilon']*df_MTO_output['rho']+1)


    return df_MTO_output


#df_test = MTO_simulator(list_rho_values=[1,2], list_eps_values=[1,2], num_MTO_per_sim=100, num_sim_per_param=5)
#df_test.sample(5)


## Epsilon

### Epsilon code

In [15]:

# general simulation stuff
list_eps_values = np.arange(0.5, 2, 0.01)

num_sim_per_param = 10 # number of simulations with given parameter set
num_MTO_per_sim = 100 # number of simulations on each network


#df_eps = MTO_simulator(list_rho_values=[1, 1.25, 1.5], list_eps_values=list_eps_values, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_eps.sample(5)
#df_eps.to_csv('eps_sim.csv')

df_eps = pd.read_csv('SimulationData/eps_sim.csv')
df_eps.sample(5)

df_eps = df_eps.rename(columns={'Network_H_Share_H': 'H_Share_H', 'Network_H_Share_L': 'H_Share_L', 'Predicted_H_Share': 'Predicted_H_Share_L'})
df_eps['Predicted_H_Share_H'] = 1-df_eps['Predicted_H_Share_L']

df_eps


,sim_id,p_SES_high,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,H_Share_H,H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share_L,Predicted_Exposure,Predicted_Friend_Bias,Predicted_H_Share_H
0,UfnqWgMQ,0.5,0.50,1.0,0.526316,1.052632,0.947368,20,19,-0.260659,0.340911,0.608753,31.90,0.358908,0.666667,1.333333,0.000000,0.333333
1,UfnqWgMQ,0.5,0.50,1.0,0.578947,1.157895,0.947368,20,19,-0.260659,0.340911,0.608753,31.90,0.358908,0.666667,1.333333,0.000000,0.333333
2,UfnqWgMQ,0.5,0.50,1.0,0.578947,1.157895,0.947368,20,19,-0.260659,0.340911,0.608753,31.90,0.358908,0.666667,1.333333,0.000000,0.333333
3,UfnqWgMQ,0.5,0.50,1.0,0.666667,1.333333,0.944444,20,18,-0.260659,0.340911,0.608753,31.90,0.358908,0.666667,1.333333,0.000000,0.333333
4,UfnqWgMQ,0.5,0.50,1.0,0.666667,1.333333,0.944444,20,18,-0.260659,0.340911,0.608753,31.90,0.358908,0.666667,1.333333,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,ARpdDwgb,0.5,1.99,1.5,0.416667,1.000000,0.940476,14,12,0.398651,0.712574,0.299282,21.16,0.267622,0.222965,0.668896,0.249686,0.777035
449996,ARpdDwgb,0.5,1.99,1.5,0.300000,1.000000,0.957143,14,10,0.398651,0.712574,0.299282,21.16,0.267622,0.222965,0.668896,0.249686,0.777035
449997,ARpdDwgb,0.5,1.99,1.5,0.250000,0.615385,0.937500,14,12,0.398651,0.712574,0.299282,21.16,0.267622,0.222965,0.668896,0.249686,0.777035
449998,ARpdDwgb,0.5,1.99,1.5,0.090909,0.571429,0.977273,14,11,0.398651,0.712574,0.299282,21.16,0.267622,0.222965,0.668896,0.249686,0.777035


### Epsilon HSES Share

In [16]:



HSESL_epsilon_rho125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.25], x_var='epsilon', y_var='H_Share_L', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=True, colors=['rgba(200,0,0,1)', 'rgba(200,0,0,1)'], legend_text='Low')

HSESH_epsilon_rho125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.25], x_var='epsilon', y_var='H_Share_H', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=True, colors=['rgba(0,0,200,1)', 'rgba(0,0,200,1)'], legend_text='High')


fig_HSES_epsilon = go.Figure(data = HSESL_epsilon_rho125.data + HSESH_epsilon_rho125.data)

fig_HSES_epsilon.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'HSES Share',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.65),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.25,0.75],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_HSES_epsilon.add_hline(y=0.5, line_width=1, annotation_text = 'Equal share')
fig_HSES_epsilon.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no bias')




### Epsilon Assortativity

In [196]:


SESAss_epsilon_rho1 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1], x_var='epsilon', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=False, colors=['rgba(250,0,250,1)'], legend_text='ρ=1')

SESAss_epsilon_rho125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.25], x_var='epsilon', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=False, colors=['rgba(150,0,150,1)'], legend_text='ρ=1.25')

SESAss_epsilon_rho15 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.5], x_var='epsilon', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=False, colors=['rgba(100,0,100,1)'], legend_text='ρ=1.5')

SESAss_epsilon_rho = go.Figure(data = SESAss_epsilon_rho15.data + SESAss_epsilon_rho125.data + SESAss_epsilon_rho1.data)

SESAss_epsilon_rho.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'SES Assortativity',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.15),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [-0.5,0.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')

SESAss_epsilon_rho.add_hline(y=0, line_width=1, annotation_text = 'Random sorting')
SESAss_epsilon_rho.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no bias')




### Epsilon Exposure DONE

In [185]:


Exposure_epsilon_rho125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.25], x_var='epsilon', y_var='Exposure', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=True, colors=['rgba(15,209,132,1)', 'rgba(15,133,84,1)'])

fig_Exposure_epsilon = go.Figure(data =Exposure_epsilon_rho125.data)

fig_Exposure_epsilon.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Exposure',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.95),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,1.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_Exposure_epsilon.add_hline(y=1, line_width=1, annotation_text = 'Even exposure')
fig_Exposure_epsilon.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no bias')


### Epsilon Friending bias DONE

In [199]:

Friending_epsilon_rho125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.25], x_var='epsilon', y_var='Friend_Bias', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=True, colors=['rgba(224,116,0,1)', 'rgba(229,187,0,1)'])



fig_Friending_epsilon = go.Figure(data = Friending_epsilon_rho125.data)

fig_Friending_epsilon.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Friending Effect',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.85),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0,1],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_Friending_epsilon.add_hline(y=0.5, line_width=1, annotation_text = 'No preference')
fig_Friending_epsilon.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no bias')


## Rho

### Rho code

In [5]:

# general simulation stuff
list_rho_values = np.arange(0.5, 2, 0.01)


num_sim_per_param = 10 # number of simulations with given parameter set
num_MTO_per_sim = 100 # number of simulations on each network


#df_rho = MTO_simulator(list_eps_values=[1, 1.25, 1.5], list_rho_values=list_rho_values, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_rho.to_csv('rho_sim.csv')

df_rho = pd.read_csv('SimulationData/rho_sim.csv')

df_rho.sample(5)


,Unnamed: 0,sim_id,p_SES_high,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,Network_H_Share_H,Network_H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share,Predicted_Exposure,Predicted_Friend_Bias
36537,36537,VxXeVszi,0.5,1.25,0.62,0.428571,0.857143,0.928571,14,14,0.104643,0.505339,0.399165,23.92,0.286637,0.716846,0.888889,-0.267606
9163,9163,IKSNEwuu,0.5,1.00,0.53,0.769231,1.538462,0.923077,14,13,-0.031938,0.490710,0.529436,24.08,0.298401,0.943396,1.000000,-0.307190
439146,439146,mbIaACdC,0.5,1.50,1.96,0.500000,1.285714,0.944444,14,10,0.428820,0.808806,0.357089,20.10,0.269579,0.204082,0.800000,0.365482
375811,375811,kNQamQuB,0.5,1.25,1.75,0.461538,1.000000,0.934066,14,13,0.324008,0.691950,0.356887,20.18,0.259676,0.253968,0.888889,0.294118
138043,138043,EYabZNCC,0.5,1.00,0.96,0.500000,1.000000,0.928571,14,14,-0.005824,0.497173,0.505025,24.14,0.295817,0.520833,1.000000,-0.020408


### Rho HSES Share

In [4]:



HSESL_rho_epsilon125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.25], x_var='rho', y_var='Network_H_Share_H', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(200,0,0,1)'], legend_text='High')

HSESH_rho_epsilon125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.25], x_var='rho', y_var='Network_H_Share_L', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(0,0,200,1)'], legend_text='Low')


fig_HSES_rho = go.Figure(data = HSESH_rho_epsilon125.data + HSESL_rho_epsilon125.data)

fig_HSES_rho.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Rho', 
    yaxis_title = 'HSES Share',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.65),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.25,0.75],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_HSES_rho.add_hline(y=0.5, line_width=1, annotation_text = 'Equal share')
fig_HSES_rho.add_vline(x=1, line_width=1, annotation_text = 'ρ=1: no bias')




### Rho Assortativity

In [6]:

SESAss_rho_epsilon1 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1], x_var='rho', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(250,0,250,1)'], legend_text='ϵ=1')

SESAss_rho_epsilon125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.25], x_var='rho', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(150,0,150,1)'], legend_text='ϵ=1.25')

SESAss_rho_epsilon15 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.5], x_var='rho', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(100,0,100,1)'], legend_text='ϵ=1.5')


SESAss_rho_epsilon125 = go.Figure(data = SESAss_rho_epsilon1.data + SESAss_rho_epsilon125.data + SESAss_rho_epsilon15.data)

SESAss_rho_epsilon125.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Rho', 
    yaxis_title = 'SES Assortativity',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.15),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [-0.5,0.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


SESAss_rho_epsilon125.add_hline(y=0, line_width=1, annotation_text = 'Random sorting')
SESAss_rho_epsilon125.add_vline(x=1, line_width=1, annotation_text = 'ρ=1: no bias')




### Rho Exposure

In [7]:

Exposure_rho_epsilon1 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1], x_var='rho', y_var='Exposure', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(15,209,132,1)'], legend_text='ϵ=1')

Exposure_rho_epsilon125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.25], x_var='rho', y_var='Exposure', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(15,170,100,1)'], legend_text='ϵ=1.25')

Exposure_rho_epsilon15 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.5], x_var='rho', y_var='Exposure', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(15,100,50,1)'], legend_text='ϵ=1.5')


fig_Exposure_rho = go.Figure(data = Exposure_rho_epsilon15.data + Exposure_rho_epsilon125.data + Exposure_rho_epsilon1.data)

fig_Exposure_rho.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Rho', 
    yaxis_title = 'Exposure',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.95),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,1.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_Exposure_rho.add_hline(y=1, line_width=1, annotation_text = 'Even exposure')
fig_Exposure_rho.add_vline(x=1, line_width=1, annotation_text = 'ρ=1: no bias')




### Rho Friending

In [8]:



Friending_rho_epsilon125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.25], x_var='rho', y_var='Friend_Bias', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=True, colors=['rgba(224,116,0,1)', 'rgba(229,187,0,1)'], legend_text='High')


fig_Friending_rho = go.Figure(data = Friending_rho_epsilon125.data)

fig_Friending_rho.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'rho', 
    yaxis_title = 'Friending Effect',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.65),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [-0.25,1],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_Friending_rho.add_hline(y=0, line_width=1, annotation_text = 'Equal share')
fig_Friending_rho.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no bias')




## Both rho and epsilon

### Specified values

In [2]:

# general simulation stuff
list_eps_values = np.arange(0.5, 2, 0.05)
list_rho_values = np.arange(0.5, 2, 0.05)

num_sim_per_param = 10 # number of simulations with given parameter set
num_MTO_per_sim = 100 # number of simulations on each network

#df_eps_rho = MTO_simulator(list_eps_values=list_eps_values, list_rho_values=list_rho_values, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_eps_rho_2 = MTO_simulator(list_eps_values=list_eps_values, list_rho_values=list_rho_values_2, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_eps_rho.to_csv('eps_rho_sim.csv')

df_eps_rho = pd.read_csv('SimulationData/eps_rho_sim.csv').drop(columns='Unnamed: 0')
#df_eps_rho = pd.read_csv('eps_rho_sim.csv')

df_eps_rho


,sim_id,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,Network_H_Share_H,Network_H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share,Predicted_Exposure,Predicted_Friend_Bias,p_SES_high
0,fuAuwIsk,0.50,0.50,0.473684,0.947368,0.947368,20,19,-0.280386,0.277246,0.576407,32.32,0.360496,1.333333,1.333333,-0.200000,0.5
1,fuAuwIsk,0.50,0.50,0.800000,1.600000,0.950000,20,20,-0.280386,0.277246,0.576407,32.32,0.360496,1.333333,1.333333,-0.200000,0.5
2,fuAuwIsk,0.50,0.50,0.550000,1.100000,0.950000,20,20,-0.280386,0.277246,0.576407,32.32,0.360496,1.333333,1.333333,-0.200000,0.5
3,fuAuwIsk,0.50,0.50,0.684211,1.368421,0.947368,20,19,-0.280386,0.277246,0.576407,32.32,0.360496,1.333333,1.333333,-0.200000,0.5
4,fuAuwIsk,0.50,0.50,0.722222,1.444444,0.944444,20,18,-0.280386,0.277246,0.576407,32.32,0.360496,1.333333,1.333333,-0.200000,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,qwbMKCoH,1.95,1.95,0.307692,0.714286,0.938462,14,13,0.531830,0.765058,0.222420,19.84,0.280910,0.173837,0.677966,0.385737,NaN
899996,qwbMKCoH,1.95,1.95,0.272727,0.769231,0.945455,14,11,0.531830,0.765058,0.222420,19.84,0.280910,0.173837,0.677966,0.385737,NaN
899997,qwbMKCoH,1.95,1.95,0.333333,0.857143,0.944444,14,12,0.531830,0.765058,0.222420,19.84,0.280910,0.173837,0.677966,0.385737,NaN
899998,qwbMKCoH,1.95,1.95,0.272727,0.857143,0.954545,14,11,0.531830,0.765058,0.222420,19.84,0.280910,0.173837,0.677966,0.385737,NaN


### Share of H friends: redo this and the rho and only vary rho or epsilon (with more values)

In [11]:

fig1 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps_rho[df_eps_rho['rho'] == 1], x_var='epsilon', y_var='Network_H_Share_H', x_range=[1,2], y_range=[0,2], x_title='epsilon', y_title='Exposure', with_predicted=False, colors=['rgba(255,0,0,1)'])
fig2 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps_rho, x_var='epsilon', y_var='Network_H_Share_L', x_range=[1,2], y_range=[0,2], x_title='epsilon', y_title='Exposure', with_predicted=False, colors=['rgba(0,0,255,1)'])

fig1['data'][0]['name'] = 'High SES'
fig2['data'][0]['name'] = 'Low SES'

fig = go.Figure(data = fig2.data + fig1.data)

fig.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Share of HSES friends',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=1),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.25,0.75],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig.add_hline(y=0.5, line_width=1, annotation_text = 'Equal share')
fig.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no bias')


In [10]:



fig1 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps_rho, x_var='rho', y_var='Network_H_Share_H', x_range=[1,2], y_range=[0,2], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(255,0,0,1)'])
fig2 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps_rho, x_var='rho', y_var='Network_H_Share_L', x_range=[1,2], y_range=[0,2], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(0,0,255,1)'])

fig1['data'][0]['name'] = 'High SES'
fig2['data'][0]['name'] = 'Low SES'

fig = go.Figure(data = fig2.data + fig1.data)

fig.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Rho', 
    yaxis_title = 'Share of HSES friends',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=1),
    legend_title_text = '',
    xaxis_range = [1,2],
    yaxis_range = [0.25,0.75],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig.add_hline(y=0.5, line_width=1, annotation_text = 'Equal share')
fig.add_vline(x=1, line_width=1, annotation_text = 'rho=1: no bias')




### SES Assortativity

In [11]:

fig_eps = plotly_lines.line_with_error(plotly_lines(), data=df_eps_rho, x_var='epsilon', y_var='Network_SES_assortativity', color='rgba(100,0,100,1)', legend_text='')

fig_eps.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'SES Assortativity',
    width = 500, height = 500,
    showlegend = False, 
    legend=dict(x=0.75,y=1),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [-0.5,0.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')

#fig_eps.data[0].line.color = "rgb(0,100,0,1)"
fig_eps.add_hline(y=0, line_width=1, annotation_text = 'Random mixing')
fig_eps.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no bias')




In [26]:

fig_rho = plotly_lines.line_with_error(plotly_lines(), data=df_eps_rho, x_var='rho', y_var='Network_SES_assortativity', color='rgba(100,0,100,1)', legend_text='')

fig_rho.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Rho', 
    yaxis_title = 'SES Assortativity',
    width = 500, height = 500,
    showlegend = False, 
    legend=dict(x=0.75,y=1),
    legend_title_text = '',
    xaxis_range = [1,2],
    yaxis_range = [-0.5,0.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_rho.add_hline(y=0, line_width=1, annotation_text = 'Random mixing')
fig_rho.add_vline(x=1, line_width=1, annotation_text = 'ρ=1: no bias')




### Exposure effect

In [ ]:


fig_pred_exposure = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Predicted_Exposure'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.sequential.RdBu_r, range_color=(0.6,1.4))

fig_pred_exposure.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 800, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'Mean<br>Exposure',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_pred_exposure.add_hline(y=1, line_width=1, annotation_text = 'ρ=1: no friending bias')
fig_pred_exposure.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no exposure bias')




In [ ]:

fig_exposure = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Exposure'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.sequential.RdBu_r)

fig_exposure.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 800, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Friending Bias'),
    coloraxis_colorbar_title_text = 'Mean<br>Friending<br>Effect',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_exposure.add_hline(y=1, line_width=1, annotation_text = 'ρ=1: no friending bias  ', annotation_font_size=20, annotation_font_color="white")
fig_exposure.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no exposure bias   ', annotation_font_size=20, annotation_font_color="white")


### Friending Effect

In [ ]:


fig_pred_friending = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Predicted_Friend_Bias'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.diverging.RdYlBu_r, range_color=(-0.5,0.5))

fig_pred_friending.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 800, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'Mean<br>Exposure',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_pred_friending.add_hline(y=1, line_width=1, annotation_text = 'ρ=1: no friending bias')
fig_pred_friending.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no exposure bias')




In [ ]:


fig_friending = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Friend_Bias'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.diverging.RdYlBu_r)

fig_friending.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 800, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'Mean<br>Exposure',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_friending.add_hline(y=1, line_width=1, annotation_text = 'ρ=1: no friending bias')
fig_friending.add_vline(x=1, line_width=1, annotation_text = 'ϵ=1: no exposure bias')




### Others?

In [8]:
px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['H_Share'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.sequential.Greens)

NameError: name 'df_eps_rho' is not defined

## Randomly sampled values

In [2]:

list_eps_values_random = np.random.normal(loc=1.1, scale=0.3, size=[10])
list_rho_values_random = np.random.normal(loc=1.1, scale=0.1, size=[10])

num_sim_per_param = 1 # number of simulations with given parameter set
num_MTO_per_sim = 50 # number of simulations on each network

df_rho_eps = MTO_simulator(list_eps_values=list_eps_values_random, list_rho_values=list_rho_values_random, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
df_rho_eps.sample(5)


Running 5000 simulations over 100 parameter combinations
 Finished 5000 simulations in 1.69 minutes, average time: 0.000338 min/sim

,sim_id,p_SES_high,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,Network_H_Share_H,Network_H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share,Predicted_Exposure,Predicted_Friend_Bias
3938,PioXsrhM,0.5,1.146198,1.018463,0.615385,1.230769,0.923077,14,13,0.024510,0.434923,0.409328,24.16,0.295267,0.457494,0.931880,0.009764
4861,UgeUzwMb,0.5,1.168129,0.950575,0.230769,0.461538,0.923077,14,13,0.064136,0.525026,0.459174,24.42,0.292484,0.485209,0.922454,-0.027357
3201,kqlPdREJ,0.5,1.108535,1.041340,0.357143,0.714286,0.928571,14,14,0.067729,0.510370,0.441027,23.92,0.292147,0.455435,0.948526,0.021272
537,nHajByjJ,0.5,0.717291,1.027146,0.571429,1.142857,0.928571,14,14,-0.139457,0.466883,0.611711,24.08,0.290593,0.566923,1.164625,0.011212
3752,mUkidFhH,0.5,1.146198,1.139185,0.500000,1.000000,0.928571,14,14,0.094537,0.625529,0.524474,23.10,0.283844,0.409012,0.931880,0.069190


### Descriptive statistics

How does the variance change with rho and epsilon?

Would appear that variance in Exposure and Friend bias increases some, as does degree. Vriance in H_Share might be going down(although need to test thsi)

I can run regressions on thisssss

### Regressions

In [3]:

df_rho_eps.groupby(['epsilon', 'rho']).var()
#df_test = df_results[['Predicted_H_Share', 'H_Share', 'Predicted_Exposure', 'Exposure', 'Predicted_Friend_Bias', 'Friend_Bias', 'Degree', 'N_exposure']].describe()



p_SES_high   H_Share  Exposure  Friend_Bias  N_exposure  \
epsilon  rho                                                                 
0.585438 0.950575         0.0  0.009206  0.036823     0.000011         0.0   
         0.979175         0.0  0.017794  0.071175     0.000023         0.0   
         1.018463         0.0  0.016220  0.064143     0.000014         0.0   
         1.027146         0.0  0.012896  0.053308     0.000016         0.0   
         1.041340         0.0  0.014763  0.054338     0.000026         0.0   
...                       ...       ...       ...          ...         ...   
1.445301 1.098516         0.0  0.013270  0.056260     0.000046         0.0   
         1.101921         0.0  0.013851  0.062114     0.000038         0.0   
         1.139185         0.0  0.021214  0.079809     0.000050         0.0   
         1.143984         0.0  0.013919  0.060916     0.000048         0.0   
         1.156695         0.0  0.021858  0.090642     0.000064         0.0   

                     Degree  Network_SES_assortativity  Network_H_Share_H  \
epsilon  rho                                                                
0.585438 0.950575  0.336327                        0.0                0.0   
         0.979175  0.630204                        0.0                0.0   
         1.018463  0.455510                        0.0                0.0   
         1.027146  0.622449                        0.0                0.0   
         1.041340  0.654694                        0.0                0.0   
...                     ...                        ...                ...   
1.445301 1.098516  0.762857                        0.0                0.0   
         1.101921  0.803673                        0.0                0.0   
         1.139185  0.640408                        0.0                0.0   
         1.143984  1.112245                        0.0                0.0   
         1.156695  0.877143                        0.0                0.0   

                   Network_H_Share_L  Network_Degree  Network_Clustering  \
epsilon  rho                                                               
0.585438 0.950575                0.0             0.0                 0.0   
         0.979175                0.0             0.0                 0.0   
         1.018463                0.0             0.0                 0.0   
         1.027146                0.0             0.0                 0.0   
         1.041340                0.0             0.0                 0.0   
...                              ...             ...                 ...   
1.445301 1.098516                0.0             0.0                 0.0   
         1.101921                0.0             0.0                 0.0   
         1.139185                0.0             0.0                 0.0   
         1.143984                0.0             0.0                 0.0   
         1.156695                0.0             0.0                 0.0   

                   Predicted_H_Share  Predicted_Exposure  \
epsilon  rho                                               
0.585438 0.950575                0.0                 0.0   
         0.979175                0.0                 0.0   
         1.018463                0.0                 0.0   
         1.027146                0.0                 0.0   
         1.041340                0.0                 0.0   
...                              ...                 ...   
1.445301 1.098516                0.0                 0.0   
         1.101921                0.0                 0.0   
         1.139185                0.0                 0.0   
         1.143984                0.0                 0.0   
         1.156695                0.0                 0.0   

                   Predicted_Friend_Bias  
epsilon  rho                              
0.585438 0.950575                    0.0  
         0.979175                    0.0  
         1.018463                    0.0  
         1.027146

In [4]:

result = sm.ols(formula="H_Share ~ epsilon + rho", data=df_rho_eps).fit()
print(result.params)
print(result.summary())


Intercept    0.777015
epsilon     -0.220436
rho         -0.059284
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                H_Share   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     517.2
Date:                Mon, 01 May 2023   Prob (F-statistic):          7.02e-205
Time:                        11:45:45   Log-Likelihood:                 2855.5
No. Observations:                5000   AIC:                            -5705.
Df Residuals:                    4997   BIC:                            -5685.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------

### Predicted versus realised

In [5]:

result = sm.ols(formula="H_Share ~ Predicted_H_Share", data=df_rho_eps).fit()
print(result.params)
print(result.summary())

#px.scatter(df_rho_eps, x='Predicted_Friend_Bias', y='Friend_Bias', height=400, width=400)


Intercept            0.108410
Predicted_H_Share    0.797058
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                H_Share   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     912.2
Date:                Mon, 01 May 2023   Prob (F-statistic):          3.23e-184
Time:                        11:45:57   Log-Likelihood:                 2804.2
No. Observations:                5000   AIC:                            -5604.
Df Residuals:                    4998   BIC:                            -5591.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------

In [5]:

result = sm.ols(formula="Exposure ~ Predicted_Exposure", data=df_rho_eps).fit()
print(result.params)
print(result.summary())

#px.scatter(df_rho_eps, x='Predicted_Friend_Bias', y='Friend_Bias', height=400, width=400)


Intercept             0.249063
Predicted_Exposure    0.749443
dtype: float64
                            OLS Regression Results                            
Dep. Variable:               Exposure   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     1302.
Date:                Sat, 29 Apr 2023   Prob (F-statistic):          2.41e-271
Time:                        20:17:36   Log-Likelihood:                -1407.3
No. Observations:               12500   AIC:                             2819.
Df Residuals:                   12498   BIC:                             2834.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

In [6]:

result = sm.ols(formula="Friend_Bias ~ Predicted_Friend_Bias", data=df_rho_eps).fit()
print(result.params)
print(result.summary())

#px.scatter(df_rho_eps, x='Predicted_Friend_Bias', y='Friend_Bias', height=400, width=400)


Intercept                0.925382
Predicted_Friend_Bias    0.089281
dtype: float64
                            OLS Regression Results                            
Dep. Variable:            Friend_Bias   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     339.3
Date:                Sat, 29 Apr 2023   Prob (F-statistic):           8.68e-75
Time:                        20:17:40   Log-Likelihood:                 31285.
No. Observations:               12500   AIC:                        -6.257e+04
Df Residuals:                   12498   BIC:                        -6.255e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------

### Sample rho and epsilon from the distribution, run MTO 100 times per simulation, and vary w_H: this shouild hopefully show how important it is to consider friendingf bias 